# 使用Tensorflow低级API建立softmax regression模型

以机器学习领域中的Hello World任务-MNIST手写数字识别来探索Tensorflow。<br>
MNIST是一个非常简单的视觉数据集，如下图所示<br>
<img src="./images/04_01.png"><br>

它也包含每一张图片对应的标签，告诉我们这个是数字几。比如，上面这四张图片的标签分别是5，0，4，1。

我们将训练一个机器学习模型用于预测图片里面的数字。

我们的目的是要介绍下如何使用TensorFlow。所以，我们这里会从一个很简单的数学模型开始，建立一个softmax regression。

对应这个教程的实现代码很短，而且真正有意思的内容只包含在三行代码里面。

但是，去理解包含在这些代码里面的设计思想是非常重要的：TensorFlow工作流程和机器学习的基本概念。

因此，这个教程会很详细地介绍这些代码的实现原理。

# MNIST数据集
MNIST数据集的官网是Yann LeCun's website。在这里，提供了一份python源代码用于自动下载和安装这个数据集。

可以下载这份代码，然后用下面的代码导入到你的项目里面，也可以直接复制粘贴到你的代码文件里面。

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.


URLError: <urlopen error [Errno 101] Network is unreachable>

下载下来的数据集被分成两部分：60000行的训练数据集（mnist.train）和10000行的测试数据集（mnist.test）。


每一张图片包含28像素X28像素。我们可以用一个数字数组来表示这张图片：
<img src="./images/04_02.png"><br>

相对应的MNIST数据集的标签是介于0到9的数字，用来描述给定图片里表示的数字。

我们使标签数据是"one-hot vectors"。 一个one-hot向量除了某一位的数字是1以外其余各维度数字都是0。

所以数字n将表示成一个只有在第n维度（从0开始）数字为1的10维向量。

比如，标签0将表示成([1,0,0,0,0,0,0,0,0,0,0])。

因此， mnist.train.labels 是一个 [60000, 10] 的数字矩阵。
<img src="./images/04_03.png"><br>

# 单层感知机
我们知道MNIST的每一张图片都表示一个数字，从0到9。我们希望得到给定图片代表每个数字的概率。

比如说，我们的模型可能推测一张包含9的图片代表数字9的概率是80%, 但是判断它是8的概率是5%（因为8和9都有上半部分的小圆），然后给予它代表其他数字的概率更小的值。

这是一个使用softmax回归（softmax regression）模型的经典案例。

softmax模型可以用来给不同的对象分配概率。即使在之后，我们训练更加精细的模型时，最后一步也需要用softmax来分配概率。

softmax回归（softmax regression）分两步：第一步

为了得到一张给定图片属于某个特定数字类的证据（evidence），我们对图片像素值进行加权求和。如果这个像素具有很强的证据说明这张图片不属于该类，那么相应的权值为负数，相反如果这个像素拥有有利的证据支持这张图片属于这个类，那么权值是正数。

下面的图片显示了一个模型学习到的图片上每个像素对于特定数字类的权值。红色代表负数权值，蓝色代表正数权值。
<img src="./images/04_04.png"><br>

我们也需要加入一个额外的偏置量（bias），因为输入往往会带有一些无关的干扰量。因此对于给定的输入图片 x 它代表的是数字 i 的证据可以表示为
<img src="./images/04_05.png"><br>

其中代表权重，$ W_{i} $代表权重，$ b_i $代表数字 i 类的偏置量。j 代表给定图片 x 的像素索引用于像素求和。然后用softmax函数可以把这些证据转换成概率 y：

<img src="./images/04_06.png"><br>

这里的softmax可以看成是一个激励（activation）函数或者链接（link）函数，把我们定义的线性函数的输出转换成我们想要的格式，也就是关于10个数字类的概率分布。因此，给定一张图片，它对于每一个数字的吻合度可以被softmax函数转换成为一个概率值。Softmax可以定义为

<img src="./images/04_07.png"><br>

对于softmax回归模型可以用下面的图解释，对于输入的xs加权求和，再分别加上一个偏置量，最后再输入到softmax函数中：
<img src="./images/04_08.png"><br>

如果把它写成一个等式，我们可以得到：
<img src="./images/04_09.png"><br>

我们也可以用向量表示这个计算过程：用矩阵乘法和向量相加。这有助于提高计算效率。（也是一种更有效的思考方式）
<img src="./images/04_10.png"><br>

更进一步，可以写成更加紧凑的方式：

<img src="./images/04_11.png"><br>

# 实现回归模型
Tensorflow不单独地运行单一的复杂计算，而是让我们可以先用图描述一系列可交互的计算操作，然后全部一起在Python之外运行。（这样类似的运行方式，可以在不少的机器学习库中看到。）

使用TensorFlow之前，首先导入它：

In [3]:
import tensorflow as tf

我们通过操作符号变量来描述这些可交互的操作单元，可以用下面的方式创建一个：

In [4]:
x = tf.placeholder(tf.float32, [None, 784])

`x`不是一个特定的值，而是一个占位符placeholder，我们在TensorFlow运行计算时输入这个值。

我们希望能够输入任意数量的MNIST图像，每一张图展平成784维的向量。

我们用2维的浮点数张量来表示这些图，这个张量的形状是[None，784 ]。（这里的None表示此张量的第一个维度可以是任何长度的。）

我们的模型也需要权重值和偏置量，当然我们可以把它们当做是另外的输入（使用占位符），但TensorFlow有一个更好的方法来表示它们：Variable 。 

一个Variable代表一个可修改的张量，存在在TensorFlow的用于描述交互性操作的图中。

它们可以用于计算输入值，也可以在计算中被修改。对于各种机器学习应用，一般都会有模型参数，可以用Variable表示。

In [5]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Instructions for updating:
Colocations handled automatically by placer.


我们赋予tf.Variable不同的初值来创建不同的Variable：在这里，我们都用全为零的张量来初始化W和b。因为我们要学习W和b的值，它们的初值可以随意设置。

注意，W的维度是[784，10]，因为我们想要用784维的图片向量乘以它以得到一个10维的证据值向量，每一位对应不同数字类。b的形状是[10]，所以我们可以直接把它加到输出上面。

现在，我们可以实现我们的模型了。只需要一行代码。

In [6]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

首先，我们用tf.matmul(​​X，W)表示x乘以W，对应之前等式里面的，这里x是一个2维张量拥有多个输入。然后再加上b，把和输入到tf.nn.softmax函数里面。

至此，我们先用了几行简短的代码来设置变量，然后只用了一行代码来定义我们的模型。

一旦被定义好之后，我们的模型就可以在不同的设备上运行：计算机的CPU，GPU，甚至是手机。

# 训练模型
为了训练我们的模型，我们首先需要定义一个指标来评估这个模型是好的。

在机器学习，我们通常定义指标来表示一个模型坏的程度，这个指标称为成本（cost）或损失（loss），然后尽量最小化这个指标。

一个非常常见的loss 函数是“交叉熵”（cross-entropy）。

交叉熵产生于信息论里面的信息压缩编码技术，但是它后来演变成为从博弈论到机器学习等其他领域里的重要技术手段。它的定义如下：
<img src="./images/04_12.png"><br>

y 是我们预测的概率分布, y' 是实际的分布（我们输入的one-hot vector)。

比较粗糙的理解是，交叉熵是用来衡量我们的预测用于描述真相的低效性。

为了计算交叉熵，我们首先需要添加一个新的占位符用于输入正确值：


In [6]:
y_ = tf.placeholder(tf.float32, [None, 10])

然后我们下面的公式计算交叉熵:

<img src="./images/04_13.png"><br>


In [7]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

首先，用`tf.log` 计算 y 的每个元素的对数。

接下来，我们把 y_ 的每一个元素和 `tf.log(y_)` 的对应元素相乘。最后，用 `tf.reduce_sum` 计算张量的所有元素的总和。（注意，这里的交叉熵不仅仅用来衡量单一的一对预测和真实值，而是所有100幅图片的交叉熵的总和。对于100个数据点的预测表现比单一数据点的表现能更好地描述我们的模型的性能。

现在我们知道我们需要我们的模型做什么了，用TensorFlow来训练它是非常容易的。

因为TensorFlow拥有一张描述你各个计算单元的图，它可以自动地使用反向传播算法(backpropagation algorithm)来有效地确定你的变量是如何影响你想要最小化的那个成本值的。

然后，TensorFlow会用你选择的优化算法来不断地修改变量以降低成本。

In [8]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

Instructions for updating:
Use tf.cast instead.


在这里，我们要求TensorFlow用梯度下降算法（gradient descent algorithm）以0.01的学习速率最小化交叉熵。

梯度下降算法（gradient descent algorithm）是一个简单的学习过程，TensorFlow只需将每个变量一点点地往使成本不断降低的方向移动。

当然TensorFlow也提供了其他许多优化算法：只要简单地调整一行代码就可以使用其他的算法。

TensorFlow在这里实际上所做的是，它会在后台给描述你的计算的那张图里面增加一系列新的计算操作单元用于实现反向传播算法和梯度下降算法。

然后，它返回给你的只是一个单一的操作，当运行这个操作时，它用梯度下降算法训练你的模型，微调你的变量，不断减少成本。

现在，我们已经设置好了我们的模型。在运行计算之前，我们需要添加一个操作来初始化我们创建的变量：


In [9]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


现在我们可以在一个`Session`里面启动我们的模型，并且初始化变量：

In [37]:
sess = tf.Session()
with sess.as_default():
    sess.run(init)
    # 然后开始训练模型，这里我们让模型循环训练1000次。
    for _ in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

该循环的每个步骤中，我们都会随机抓取训练数据中的100个批处理数据点，然后我们用这些数据点作为参数替换之前的占位符来运行train_step。

使用一小部分的随机数据来进行训练被称为随机训练（stochastic training）- 在这里更确切的说是随机梯度下降训练。

在理想情况下，我们希望用我们所有的数据来进行每一步的训练，因为这能给我们更好的训练结果，但显然这需要很大的计算开销。

所以，每一次训练我们可以使用不同的数据子集，这样做既可以减少计算开销，又可以最大化地学习到数据集的总体特性。

# 评估我们的模型
那么我们的模型性能如何呢？

首先让我们找出那些预测正确的标签。
`tf.argmax` 是一个非常有用的函数，它能给出某个tensor对象在某一维上的其数据最大值所在的索引值。
由于标签向量是由0,1组成，因此最大值1所在的索引位置就是类别标签，比如tf.argmax(y,1)返回的是模型对于任一输入x预测到的标签值，
而 tf.argmax(y_,1) 代表正确的标签，我们可以用 tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)。

In [38]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

这行代码会给我们一组布尔值。为了确定正确预测项的比例，我们可以把布尔值转换成浮点数，然后取平均值。例如，[True, False, True, True] 会变成 [1,0,1,1] ，取平均值后得到 0.75.

In [39]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

最后，我们计算所学习到的模型在测试数据集上面的正确率。

In [40]:
with sess.as_default():
    print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

0.8847


这个结果好吗？嗯，并不太好。事实上，这个结果是很差的。这是因为我们仅仅使用了一个非常简单的模型。不过，做一些小小的改进，我们就可以得到97％的正确率。最好的模型甚至可以获得超过99.7％的准确率